## EVALUATION OF THE MODELS

In [ ]:
import argparse
import pickle
from nuscenes import NuScenes
import os
import json
from nuscenes.map_expansion.map_api import NuScenesMap

from pyquaternion import Quaternion
from nuscenes.utils.geometry_utils import transform_matrix
from nuscenes import NuScenes
from nuscenes.prediction import PredictHelper
from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.compute_metrics import compute_metrics
import torch
import numpy as np
from pyquaternion import Quaternion
from nuscenes.eval.prediction.data_classes import Prediction
from nuscenes.eval.prediction.compute_metrics import compute_metrics
from nuscenes.prediction.models.physics import ConstantVelocityHeading, PhysicsOracle
from nuscenes.eval.prediction.data_classes import Prediction

from nuscenes import NuScenes
from nuscenes.prediction import PredictHelper
from nuscenes.eval.prediction.data_classes import Prediction
from nuscenes.eval.prediction.metrics import MinADEK, MinFDEK, RowMean
from nuscenes.prediction.models.backbone import ResNetBackbone

from matplotlib import pyplot as plt

from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.compute_metrics import compute_metrics
from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer

from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
from nuscenes.prediction.models.physics import ConstantVelocityHeading, PhysicsOracle

from nuscenes.map_expansion import arcline_path_utils
from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.models.mtp import MTP
from nuscenes.prediction.models.covernet import CoverNet

from torch.utils.data import Dataset, DataLoader
from nuscenes.prediction.models.mtp import MTPLoss

In [ ]:
from src.dataset import NuScenesMTPInferenceDataset

def generate_submission_notebook(model, dataset, output_path="submission.json"):
    model.eval()
    predictions_list = []
    
    # Necesitamos el helper para buscar la pose del agente
    helper = dataset.helper 

    print(f"🚗 Generando submission con conversión LOCAL -> GLOBAL...")
    
    for i in tqdm(range(len(dataset))):
        img, agent_state, _, _ = dataset[i]
        
        # Recuperar tokens
        raw_token = dataset.split[i]
        instance_token, sample_token = raw_token.split("_")

        # Inferencia
        img = img.unsqueeze(0)        
        agent_state = agent_state.unsqueeze(0)
        with torch.no_grad():
            pred = model(img, agent_state)

        # Procesar salida (tu código de antes)
        total_output_size = pred.shape[1]
        num_modes = total_output_size // 25 
        num_coords = num_modes * 24
        
        pred_coords = pred[0, :num_coords]
        pred_probs = pred[0, num_coords:]
        
        # [Num_modos, 12, 2] en coordenadas LOCALES
        pred_coords_local = pred_coords.reshape(num_modes, 12, 2).cpu().numpy()

        # ============================================================
        # 🌍 TRANSFORMACIÓN CRÍTICA: LOCAL -> GLOBAL
        # ============================================================
        
        # 1. Obtener la pose actual del agente en el mapa global
        sample_annotation = helper.get_sample_annotation(instance_token, sample_token)
        translation = sample_annotation['translation'] # [x, y, z] global
        rotation = sample_annotation['rotation']       # Quaternion global
        
        # 2. Convertir a matriz de transformación (Local -> Global)
        # Nota: transform_matrix espera rotación como Quaternion y translación
        # Pero ojo: MTP predice X,Y (2D). NuScenes es 3D.
        
        # Manera simplificada de rotar y trasladar vectores 2D:
        quaternion = Quaternion(rotation)
        
        # Creamos un array vacío para las coordenadas globales
        pred_coords_global = np.zeros_like(pred_coords_local)

        for mode_idx in range(num_modes):
            # Cogemos la trayectoria de un modo (Shape: 12, 2)
            trajectory_local = pred_coords_local[mode_idx]
            
            # A. Añadimos una columna de ceros para Z (necesario para rotación 3D)
            # Shape se convierte en (12, 3) -> [x, y, 0]
            traj_3d = np.hstack([trajectory_local, np.zeros((12, 1))])
            
            # B. Rotar (El agente mira hacia una dirección, rotamos los puntos)
            # Iteramos punto a punto o usamos vectorización si es posible. 
            # rotate funciona con vector único, así que iteramos para asegurar:
            traj_rotated = np.array([quaternion.rotate(p) for p in traj_3d])
            
            # C. Trasladar (Sumar la posición global actual del coche)
            # Solo sumamos X e Y (índices 0 y 1)
            pred_coords_global[mode_idx, :, 0] = traj_rotated[:, 0] + translation[0]
            pred_coords_global[mode_idx, :, 1] = traj_rotated[:, 1] + translation[1]
            
        # ============================================================

        # Probabilidades
        if num_modes > 1:
            probs = torch.nn.functional.softmax(pred_probs, dim=0).cpu().numpy()
        else:
            probs = np.array([1.0])

        prediction_obj = Prediction(
            instance=instance_token,
            sample=sample_token,
            prediction=pred_coords_global, # ¡USAMOS LAS GLOBALES!
            probabilities=probs
        )

        predictions_list.append(prediction_obj.serialize())

    with open(output_path, "w") as f:
        json.dump(predictions_list, f, indent=2)

    return output_path

The function `evaluate_submission` loads a submission file, runs the official
nuScenes evaluation pipeline, and stores the resulting metrics. Keeping this
code inside `src/` allows us to evaluate any trained model with a single
function call from our notebook.

In [ ]:
from src.evaluate import evaluate_submission
from src.evaluate import load_model 


# IMPORTANTE: primero defines el modelo y el optimizador vacíos
backbone = ResNetBackbone('resnet50')
model = MTP(backbone, num_modes=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Ahora cargas pesos + optimizer
model, optimizer, epoch = load_model(model, optimizer,
                                     filename="mtp_lane_loss_v1.pth",
                                     device=device)

In [ ]:
mini_val = get_prediction_challenge_split("mini_val", dataroot=DATAROOT)
dataset_val = NuScenesMTPInferenceDataset(mini_val, helper, mtp_input_representation, device=device)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=4, shuffle=False)
len(mini_val)

# Generar submission
submission_path = generate_submission_notebook(model, dataset_val, "submission_mtp.json")

In [ ]:
metrics = evaluate_submission("v1.0-mini", DATAROOT, submission_path)

## VISUALIZATION OF THE RESULTS

In [ ]:
# --- FUNCIONES AUXILIARES ---
def meters_to_pixels(coords_meters, img_size=500, resolution=0.2, center_offset=(250, 250)):
    """
    Convierte coordenadas locales (metros) a coordenadas de imagen (píxeles).
    Asume que el agente está en el centro de la imagen mirando hacia ARRIBA.
    
    Coords input: [Lateral (X), Adelante (Y)] según descubrimos en tu modelo.
    """
    # Desempaquetamos
    lateral_meters = coords_meters[:, 0]
    forward_meters = coords_meters[:, 1]
    
    center_x, center_y = center_offset
    
    # Conversión a píxeles
    # X en imagen aumenta hacia la derecha. Lateral positivo es derecha.
    pix_x = center_x + (lateral_meters / resolution)
    
    # Y en imagen aumenta hacia ABAJO. Forward positivo es ARRIBA (restamos).
    pix_y = center_y - (forward_meters / resolution)
    
    return pix_x, pix_y

In [ ]:
# 1. Usamos los tokens y datos del caso de Overfitting
# (Asegúrate de que estas variables instance_token y sample_token siguen definidas de la celda anterior)
print(f"Visualizando Agente: {instance_token} en Sample: {sample_token}")

# 2. Obtener la IMAGEN BASE (Rasterizada)
# Usamos el código que tú propusiste
img_raster = mtp_input_representation.make_input_representation(instance_token, sample_token)

# 3. Obtener REALIDAD LOCAL (Ground Truth en metros, desde el helper)
gt_local_meters = helper.get_future_for_agent(instance_token, sample_token, seconds=6, in_agent_frame=True)

# 4. Obtener PREDICCIÓN LOCAL (Del modelo overfiteado)
# Recuperamos los datos crudos del dataset para inferencia
img_tensor, agent_tensor, _ = dataset_train[idx] # Usamos el mismo idx=0 del overfitting

model.eval()
with torch.no_grad():
    # Inferencia
    pred_raw = model(img_tensor.unsqueeze(0).to(device), agent_tensor.unsqueeze(0).to(device))

# Desempaquetar (Asumimos 1 modo principal ya que está overfiteado)
num_modes = pred_raw.shape[1] // 25
pred_coords_all = pred_raw[0, :num_modes*24].reshape(num_modes, 12, 2).cpu().numpy()
pred_local_meters = pred_coords_all[0] # Tomamos el primer modo

# 5. CONVERTIR A PÍXELES
# Importante: el rasterizador estándar suele usar resolución 0.2m/px (100m en 500px)
gt_pix_x, gt_pix_y = meters_to_pixels(gt_local_meters, resolution=0.2)
pred_pix_x, pred_pix_y = meters_to_pixels(pred_local_meters, resolution=0.2)

print("Datos preparados. Listos para visualizar.")

plt.figure(figsize=(10, 10))

# 1. Mostrar la imagen rasterizada de fondo
plt.imshow(img_raster)

# 2. Pintar el Agente (punto central)
plt.plot(250, 250, 'yo', markersize=12, markeredgecolor='k', label='Agente (Inicio)')

# 3. Pintar Realidad (Verde grueso)
plt.plot(gt_pix_x, gt_pix_y, 'g.-', linewidth=4, markersize=8, alpha=0.7, label='Realidad (GT)')

# 4. Pintar Predicción (Rojo discontinuo encima)
plt.plot(pred_pix_x, pred_pix_y, 'r.--', linewidth=3, markersize=8, label='Predicción Modelo')

plt.legend()
plt.title("Vista de Pájaro Local: Predicción vs Realidad (Overfitting)\n(Si sale perfecto, las líneas se solapan)")
plt.axis('off')
plt.show()

Visualization but with video instead of foto: 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(img_raster)
ax.axis('off')
ax.set_title("Animación Local: Predicción (Rojo) vs Realidad (Verde)")

# Inicializar elementos vacíos
line_gt, = ax.plot([], [], 'g-', linewidth=3, alpha=0.6, label='Traza Real')
point_gt, = ax.plot([], [], 'go', markersize=10, label='Posición Real')

line_pred, = ax.plot([], [], 'r--', linewidth=2, label='Traza Predicción')
point_pred, = ax.plot([], [], 'ro', markersize=10, label='Posición Predicha')

ax.legend(loc='upper left')

def init():
    line_gt.set_data([], [])
    point_gt.set_data([], [])
    line_pred.set_data([], [])
    point_pred.set_data([], [])
    return line_gt, point_gt, line_pred, point_pred

def update(frame):
    # Frame va de 0 a 11 (los 12 pasos)
    
    # Actualizar Realidad
    line_gt.set_data(gt_pix_x[:frame+1], gt_pix_y[:frame+1])
    point_gt.set_data([gt_pix_x[frame]], [gt_pix_y[frame]])
    
    # Actualizar Predicción
    line_pred.set_data(pred_pix_x[:frame+1], pred_pix_y[:frame+1])
    point_pred.set_data([pred_pix_x[frame]], [pred_pix_y[frame]])
    
    return line_gt, point_gt, line_pred, point_pred

# Crear animación (interval=300ms para que vaya lento y se vea bien)
anim = FuncAnimation(fig, update, frames=len(gt_pix_x), init_func=init, blit=True, interval=300)

plt.close() # Cierra la figura estática para no duplicar
HTML(anim.to_jshtml())